#### **1. Importing Libraries**

In [104]:
import pandas as pd                                                 # Importing for panel data analysis
#-------------------------------------------------------------------------------------------------------------------------------
import numpy as np                                                  # Importing package numpys (For Numerical Python)
#-------------------------------------------------------------------------------------------------------------------------------
import matplotlib.pyplot as plt                                     # Importing pyplot interface of matplotlib
import seaborn as sns                                               # Importing seaborn library for interactive visualization
%matplotlib inline
import os
import copy
import pickle
import math
import time
#-------------------------------------------------------------------------------------------------------------------------------
import warnings                                                     # Importing warning to disable runtime warnings
warnings.filterwarnings("ignore")                                   # Warnings will appear only once

In [105]:
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
print('Shape of the dataset:', ratings.shape)
ratings.head(5)

Shape of the dataset: (100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [106]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [107]:
print("Number of unique users:", ratings["userId"].nunique())

Number of unique users: 610


In [108]:
print("Number of unique movies:", ratings["movieId"].nunique())

Number of unique movies: 9724


#### **2. Data preprocessing**

#### Form the transactional data set, which consists of entries of the form <user id, {movies rated above 2}

In [109]:
# let's extract the number of unique movies and its corresponding ratings
group = ratings.groupby('movieId')
df = group.apply(lambda x: x['rating'].unique())
df

movieId
1             [4.0, 4.5, 2.5, 3.5, 3.0, 5.0, 0.5, 2.0, 1.5]
2         [4.0, 3.0, 3.5, 4.5, 2.5, 5.0, 1.5, 1.0, 2.0, ...
3             [4.0, 5.0, 3.0, 3.5, 2.0, 1.0, 2.5, 0.5, 1.5]
4                                      [3.0, 1.0, 2.0, 1.5]
5             [5.0, 3.0, 4.0, 2.0, 3.5, 4.5, 1.5, 2.5, 0.5]
                                ...                        
193581                                                [4.0]
193583                                                [3.5]
193585                                                [3.5]
193587                                                [3.5]
193609                                                [4.0]
Length: 9724, dtype: object

- So, there are movies that have been rated 2 or less. Let's keep only entries where movie ratings are greater than 2. 

In [110]:
ratings_above_2 = ratings[ratings["rating"] > 2.0]
ratings_above_2

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [111]:
# Let's extract the number of unique movies that each user might have rated
group = ratings_above_2.groupby('userId')
df = group.apply(lambda x: len(x['movieId'].unique()))
df

userId
1       226
2        28
3        18
4       167
5        40
       ... 
606    1070
607     174
608     670
609      37
610    1233
Length: 610, dtype: int64

In [112]:
count_freq = dict(df)
count_freq

{1: 226,
 2: 28,
 3: 18,
 4: 167,
 5: 40,
 6: 294,
 7: 111,
 8: 43,
 9: 34,
 10: 119,
 11: 59,
 12: 32,
 13: 28,
 14: 42,
 15: 111,
 16: 96,
 17: 105,
 18: 493,
 19: 357,
 20: 210,
 21: 380,
 22: 70,
 23: 120,
 24: 107,
 25: 26,
 26: 19,
 27: 109,
 28: 476,
 29: 78,
 30: 34,
 31: 45,
 32: 99,
 33: 137,
 34: 67,
 35: 22,
 36: 35,
 37: 20,
 38: 63,
 39: 90,
 40: 94,
 41: 170,
 42: 353,
 43: 114,
 44: 38,
 45: 366,
 46: 42,
 47: 111,
 48: 33,
 49: 21,
 50: 236,
 51: 319,
 52: 130,
 53: 20,
 54: 31,
 55: 16,
 56: 46,
 57: 379,
 58: 103,
 59: 101,
 60: 22,
 61: 37,
 62: 357,
 63: 248,
 64: 504,
 65: 34,
 66: 337,
 67: 33,
 68: 1085,
 69: 44,
 70: 61,
 71: 30,
 72: 45,
 73: 187,
 74: 177,
 75: 51,
 76: 87,
 77: 25,
 78: 47,
 79: 60,
 80: 167,
 81: 17,
 82: 207,
 83: 95,
 84: 287,
 85: 27,
 86: 69,
 87: 20,
 88: 52,
 89: 425,
 90: 53,
 91: 495,
 92: 24,
 93: 97,
 94: 44,
 95: 160,
 96: 66,
 97: 35,
 98: 81,
 99: 44,
 100: 141,
 101: 50,
 102: 52,
 103: 362,
 104: 254,
 105: 717,
 106: 33,
 10

In [113]:
# create a new column to record the number of movies rated by each userId
ratings_above_2['count_freq_userId'] = ratings_above_2['userId']
ratings_above_2['count_freq_userId'] = ratings_above_2['count_freq_userId'].map(count_freq)
ratings_above_2

,userId,movieId,rating,timestamp,count_freq_userId
0,1,1,4.0,964982703,226
1,1,3,4.0,964981247,226
2,1,6,4.0,964982224,226
3,1,47,5.0,964983815,226
4,1,50,5.0,964982931,226
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,1233
100832,610,168248,5.0,1493850091,1233
100833,610,168250,5.0,1494273047,1233
100834,610,168252,5.0,1493846352,1233


----

- Let's keep only those users who have rated more than 10 movies.

In [114]:
# Now let's remove the rows where the value of 'count_freq_userId' is less than 10.
more_than_10_movies_rated_above_2 = ratings_above_2.drop(ratings_above_2[ratings_above_2['count_freq_userId'] <= 10].index)
print(more_than_10_movies_rated_above_2)

        userId  movieId  rating   timestamp  count_freq_userId
0            1        1     4.0   964982703                226
1            1        3     4.0   964981247                226
2            1        6     4.0   964982224                226
3            1       47     5.0   964983815                226
4            1       50     5.0   964982931                226
...        ...      ...     ...         ...                ...
100831     610   166534     4.0  1493848402               1233
100832     610   168248     5.0  1493850091               1233
100833     610   168250     5.0  1494273047               1233
100834     610   168252     5.0  1493846352               1233
100835     610   170875     3.0  1493846415               1233

[87295 rows x 5 columns]


----

- Let's create the transactional data of the form <user id, {movies rated above 2}>

In [124]:
# make a new dataframe with all unique userId
transactional_df = pd.DataFrame({'userId':more_than_10_movies_rated_above_2.userId.unique()})

# And then just get the list of all unique subreddits they are active in, assigning it to a new column
transactional_df['movies_rated_above_2'] = [set(more_than_10_movies_rated_above_2['movieId'].loc[more_than_10_movies_rated_above_2['userId'] == x['userId']]) 
    for _, x in transactional_df.iterrows()]

transactional_df

,userId,movies_rated_above_2
0,1,"{1024, 1, 1025, 3, 2048, 1029, 6, 1030, 1031, ..."
1,2,"{115713, 122882, 48516, 91529, 80906, 91658, 1..."
2,3,"{70946, 2851, 5764, 4518, 26409, 7991, 1275, 2..."
3,4,"{1025, 3079, 3083, 21, 1046, 2583, 4121, 538, ..."
4,5,"{1, 515, 261, 265, 527, 531, 21, 150, 534, 153..."
...,...,...
602,606,"{1, 8195, 6148, 7, 11, 69644, 4109, 15, 17, 18..."
603,607,"{1, 517, 2053, 2054, 1544, 3081, 11, 1036, 257..."
604,608,"{1, 4105, 10, 6157, 16, 21, 31, 32, 2080, 34, ..."
605,609,"{1, 137, 10, 650, 1161, 786, 150, 288, 161, 10..."


In [66]:
print("Number of unique users:", more_than_10_movies_rated_above_2["userId"].nunique())

Number of unique users: 607


In [14]:
print("Number of unique movies:", more_than_10_movies_rated_above_2["movieId"].nunique())

Number of unique movies: 8852


- As we observe, the number of unique users are reduced from 610 to 607 after preprocessing, and the number of unique movies have reduced from 9724 to 8852.

----

- Divide the data set into 80% training set and 20% test set. Remove 20% of
movies watched from each user and create a test set using the removed
movies

In [94]:
# dummy data
dummy_df = pd.DataFrame({'userId':[1,2,4,6,8], 'movies_rated_above_2':[[100,200,300,400,500,600,700], [100,200,300,400], [300,400,500,600,700,800], [500,600,700,800,900,1000,1100,1200], [700,800,900]]})
dummy_df

,userId,movies_rated_above_2
0,1,"[100, 200, 300, 400, 500, 600, 700]"
1,2,"[100, 200, 300, 400]"
2,4,"[300, 400, 500, 600, 700, 800]"
3,6,"[500, 600, 700, 800, 900, 1000, 1100, 1200]"
4,8,"[700, 800, 900]"


In [95]:
# dividing dummy df into 80-20 train-test, such that 20% of movies watched from each user is test set.
'''
- Parse through each user
- Randomly shuffle the items in the list and split into 80-20
- extract 20 of each user and make a separate df
'''

'\n- Parse through each user\n- Randomly shuffle the items in the list and split into 80-20\n- extract 20 of each user and make a separate df\n'

In [96]:
cols = ['userId', 'movies_rated_above_2']
train_df = pd.DataFrame(columns=cols)
test_df = pd.DataFrame(columns=cols)


# loop through the rows using iterrows()
for index, row in dummy_df.iterrows():
    # print(row['userId'], row['movies_rated_above_2'])
    print(row['movies_rated_above_2'])
    print("-----")
    n = int(np.ceil(0.2 * len(row['movies_rated_above_2'])))  # initialize a value that represents 20% of the total items in the list.
    test_list = random.sample(row['movies_rated_above_2'], n)  # randomly choose 20% of the values (n) from list and make a sublist.
    print("test_list", test_list)
    print("-----")
    train_list = [i for i in row['movies_rated_above_2'] if i not in test_list] # rest 80% values of list is in train/-list
    print("train_list", train_list) # randomly choose 20% of the values from list and make a sublist
    print("******************************************************")
    
    df_1 = pd.DataFrame({
    'userId': [row['userId']],
    'movies_rated_above_2': [train_list]
    })

    df_2 = pd.DataFrame({
    'userId': [row['userId']],
    'movies_rated_above_2': [test_list]
    })

    train_df = pd.concat([train_df, df_1])
    test_df = pd.concat([test_df, df_2])
    # print("index", index)
    # train_df.loc[index].userId = row['userId']
    # train_df.loc[index].movies_rated_above_2 = train_list

    # test_df.loc[index].userId = row['userId']
    # test_df.loc[index].movies_rated_above_2 = test_list


[100, 200, 300, 400, 500, 600, 700]
-----
test_list [500, 400]
-----
train_list [100, 200, 300, 600, 700]
******************************************************
[100, 200, 300, 400]
-----
test_list [400]
-----
train_list [100, 200, 300]
******************************************************
[300, 400, 500, 600, 700, 800]
-----
test_list [800, 700]
-----
train_list [300, 400, 500, 600]
******************************************************
[500, 600, 700, 800, 900, 1000, 1100, 1200]
-----
test_list [1000, 500]
-----
train_list [600, 700, 800, 900, 1100, 1200]
******************************************************
[700, 800, 900]
-----
test_list [700]
-----
train_list [800, 900]
******************************************************


In [99]:
dummy_df

,userId,movies_rated_above_2
0,1,"[100, 200, 300, 400, 500, 600, 700]"
1,2,"[100, 200, 300, 400]"
2,4,"[300, 400, 500, 600, 700, 800]"
3,6,"[500, 600, 700, 800, 900, 1000, 1100, 1200]"
4,8,"[700, 800, 900]"


In [97]:
train_df

,userId,movies_rated_above_2
0,1,"[100, 200, 300, 600, 700]"
0,2,"[100, 200, 300]"
0,4,"[300, 400, 500, 600]"
0,6,"[600, 700, 800, 900, 1100, 1200]"
0,8,"[800, 900]"


In [98]:
test_df

,userId,movies_rated_above_2
0,1,"[500, 400]"
0,2,[400]
0,4,"[800, 700]"
0,6,"[1000, 500]"
0,8,[700]


----

- Divide the data set into 80% training set and 20% test set. Remove 20% of
movies watched from each user and create a test set using the removed
movies

In [125]:
# extending the operations of dummy data on the original data

transactional_df

,userId,movies_rated_above_2
0,1,"{1024, 1, 1025, 3, 2048, 1029, 6, 1030, 1031, ..."
1,2,"{115713, 122882, 48516, 91529, 80906, 91658, 1..."
2,3,"{70946, 2851, 5764, 4518, 26409, 7991, 1275, 2..."
3,4,"{1025, 3079, 3083, 21, 1046, 2583, 4121, 538, ..."
4,5,"{1, 515, 261, 265, 527, 531, 21, 150, 534, 153..."
...,...,...
602,606,"{1, 8195, 6148, 7, 11, 69644, 4109, 15, 17, 18..."
603,607,"{1, 517, 2053, 2054, 1544, 3081, 11, 1036, 257..."
604,608,"{1, 4105, 10, 6157, 16, 21, 31, 32, 2080, 34, ..."
605,609,"{1, 137, 10, 650, 1161, 786, 150, 288, 161, 10..."


In [137]:
# dividing transactional_df df into 80-20 train-test, such that 20% of movies watched from each user is test set.
'''
- Parse through each user
- Randomly shuffle the items in the list and split into 80-20
- extract 20 of each user and make a separate df
'''

cols = ['userId', 'movies_rated_above_2']
train_df = pd.DataFrame(columns=cols)
test_df = pd.DataFrame(columns=cols)

# loop through the rows using iterrows()
for index, row in transactional_df.iterrows():
    # print(row['userId'], row['movies_rated_above_2'])
    # print(row['movies_rated_above_2'])
    # print("-----")
    n = int(np.ceil(0.2 * len(row['movies_rated_above_2']))) # initialize a value that represents 20% of the total items in the list.
    test_list = random.sample(row['movies_rated_above_2'], n)  # randomly choose 20% of the values (n) from list and make a sublist.
    # print("test_list", test_list)
    # print("-----")
    train_list = [i for i in row['movies_rated_above_2'] if i not in test_list] # rest 80% values of list is in train_list
    # print("train_list", train_list)
    # print("******************************************************")
    
    df_1 = pd.DataFrame({
    'userId': [row['userId']],
    'movies_rated_above_2': [train_list]
    })

    df_2 = pd.DataFrame({
    'userId': [row['userId']],
    'movies_rated_above_2': [test_list]
    })

    train_df = pd.concat([train_df, df_1])
    test_df = pd.concat([test_df, df_2])

In [138]:
train_df

,userId,movies_rated_above_2
0,1,"[1024, 1, 1025, 1029, 6, 1030, 1031, 1032, 205..."
0,2,"[115713, 122882, 48516, 80906, 91658, 131724, ..."
0,3,"[70946, 2851, 5764, 26409, 7991, 1275, 2288, 8..."
0,4,"[1025, 3079, 3083, 21, 1046, 2583, 4121, 538, ..."
0,5,"[1, 515, 261, 265, 527, 531, 150, 534, 153, 41..."
...,...,...
0,606,"[1, 8195, 11, 69644, 4109, 15, 17, 2065, 2073,..."
0,607,"[1, 517, 2054, 1544, 3081, 11, 1036, 2571, 527..."
0,608,"[1, 4105, 6157, 16, 21, 31, 2080, 34, 2083, 41..."
0,609,"[1, 137, 10, 650, 1161, 786, 150, 288, 161, 10..."


In [139]:
test_df

,userId,movies_rated_above_2
0,1,"[1136, 1196, 235, 1473, 1258, 1348, 423, 3703,..."
0,2,"[79132, 8798, 106782, 91529, 3578, 46970]"
0,3,"[3024, 5181, 6835, 4518]"
0,4,"[648, 45, 3809, 912, 595, 1885, 475, 348, 2692..."
0,5,"[364, 247, 594, 21, 344, 58, 349, 290]"
...,...,...
0,606,"[2409, 8873, 4979, 4226, 27722, 1416, 1704, 37..."
0,607,"[1387, 724, 3109, 2028, 2053, 1918, 208, 204, ..."
0,608,"[5049, 741, 2231, 6377, 837, 608, 832, 2841, 1..."
0,609,"[185, 356, 892, 339, 454, 589, 110, 318]"


In [146]:
# let's confirm if the first row of the transactional data has been split into 80-20.
print(len(transactional_df["movies_rated_above_2"].iloc[0]))
print(len(train_df["movies_rated_above_2"].iloc[0]))
print(len(test_df["movies_rated_above_2"].iloc[0]))

226
180
46


----

- Saving the 80% of training data and 20% of test data in the csv

In [147]:
train_df.to_csv('./output/transactional_df_train.csv')
test_df.to_csv('./output/transactional_df_test.csv')

In [ ]:
import pandas as pd                                                 # Importing for panel data analysis
#-------------------------------------------------------------------------------------------------------------------------------
import numpy as np                                                  # Importing package numpys (For Numerical Python)
#-------------------------------------------------------------------------------------------------------------------------------
import matplotlib.pyplot as plt                                     # Importing pyplot interface of matplotlib
import seaborn as sns                                               # Importing seaborn library for interactive visualization
%matplotlib inline
import os
import copy
import pickle
import math
import time
#-------------------------------------------------------------------------------------------------------------------------------
import warnings                                                     # Importing warning to disable runtime warnings
warnings.filterwarnings("ignore")                                   # Warnings will appear only once

<a name = Section1></a>
#### **2. Data Acquisition and Description**

Lets analyze the dataset and identify what attributes require generalization/categorization before we perform BUC on them.

In [ ]:
data = pd.read_excel(f'./data/master.xlsx') # Load the Excel dataset
print('Shape of the dataset:', data.shape)
data.head(3) 

- We have 27820 records and 12 attributes.
- In our records, we have variety of data including nominal data, binomial data, numerical data.

In [ ]:
data.info() # Display basic information about the dataset

Checking for:
1. duplicate values in rows - delete duplicate rows
2. missing values in column

In [ ]:
duplicate = data[data.duplicated()] # Selecting duplicate rows except first occurrence based on all columns
duplicate

- It means there are no duplicate records

In [ ]:
print(data.isnull().sum())  # Check for missing values

- As we can observe HDI has got 19,456 null values, out of total 27,820 entries. Given, more than half of the entries having NULL values, let's discount this column.

In [ ]:
data.drop(['HDI for year'], axis=1, inplace=True)   # Remove the mentioned column

In [ ]:
data.columns # remaining columns in our dataframe

<a name = Section2></a>
#### **3. Data Analysis and AOI**

Now, let's one by one, analyze the 11 dimensions and determine for which dimensions, we need to perform Attribute Oriented Induction (AOI) for generalization/categorization.

Data generalization summarizes data by replacing relatively low-level values with higher-level concepts, or by reducing the number of dimensions to summarize data in concept space involving fewer dimensions.

In [ ]:
print("Unique values in coloumn country:\n", data["country"].unique())
print("---------------------------------------------------------")
print("Number of unique values:", data["country"].nunique())

- We will use the values of 'country' dimension as it is, because it is already in the highest-level of concept hierarchy.

In [ ]:
print("Unique values in coloumn year:\n", data["year"].unique())
print("--------------------------------------------")
print("Number of unique values:", data["year"].nunique())

- We will use the values of 'year' dimension as it has 32 distinct values.

In [ ]:
print("Unique values in coloumn sex:\n", data["sex"].unique())
print("--------------------------------------------")
print("Number of unique values:", data["sex"].nunique())

- We will use the values of 'sex' attribute/dimension as it is because it is already generalized, having two distinct values of the dimension.

In [ ]:
print("Unique values in coloumn age:\n", data["age"].unique())
print("--------------------------------------------")
print("Number of unique values:", data["age"].nunique())

- We will use the values of 'age' dimension as it is because it is already characterized by six distinct values of the dimension.

In [ ]:
print("Unique values in coloumn suicide_no:\n", data["suicides_no"].unique())
print("--------------------------------------------")
print("Number of unique values:", data["suicides_no"].nunique())

In [ ]:
data["suicides_no"].describe()    # describe the values of the suicides_no attribute.

In [ ]:
data["suicides_no"].value_counts()

- We can derive from the describe() that, the minimum number of suicides are 0. The maximum number of suicide value is 22338.
At 25th percentile, the suicide value is 3. This means that 25 percent of data that lies below this 25th percentile point will have value equal to or less than 3.
- At 50th percentile, the suicide value is 25. This means half of the data points below 50th percentile point will have value equal to or less than 25. For the high-level description purpose, we can label all those values as `low_suicide_range`.
- At 75th percentile, the suicide value is 131. This means that 75% of the data points that lies below this 75th percentile point will have value equal to or less than 131. For the high-level description purpose, we can label all those values above `low_suicide_range` and below the value at 75th percentile as `medium_suicide_range`.
- Similarly, the maximum suicide number reported is 22338. All values that lie between 75th percentile value to the maximum reported value can be termed as `high_suicide_range`.

In [ ]:
# Define the labels and conditions
conditions = [
    (data['suicides_no'] <= data['suicides_no'].quantile(0.5)),
    (data['suicides_no'] > data['suicides_no'].quantile(0.5)) & (data['suicides_no'] <= data['suicides_no'].quantile(0.75)),
    (data['suicides_no'] > data['suicides_no'].quantile(0.75))
]

labels = ['low_suicides_range', 'medium_suicides_range', 'high_suicides_range']

# Create a new column with the labels
data['suicides_range'] = np.select(conditions, labels, default='unknown')
data.drop(['suicides_no'], axis=1, inplace=True)   # Remove the column 'suicides_no' because we are using 'suicides_range' in place of that.
# Display the first few rows of the DataFrame with the new column
data.head(2)

In [ ]:
print("Unique values in column population:\n", data["population"].unique())
print("--------------------------------------------")
print("Number of unique values:", data["population"].nunique())

- "population" can't be used directly. Need to perform AOI to create higher-level descriptions or categories for numerical data.

In [ ]:
data["population"].describe().apply(lambda x: format(x, 'f')) # Suppress Scientific Notation

In [ ]:
data["population"].value_counts()

- We can derive from the describe() that, the minimum reported population is 278. The maximum reported population is 43805214.
- At 25th percentile, the reported population value is 97498.5. This means that 25 percent of data that lies below this 25th percentile point will have value equal to or less than 97498.
- At 50th percentile, the population value is 430150. This means half of the data points below 50th percentile point will have value equal to or less than 430150.
- At 75th percentile, the population value is 1486143.25. This means that 75% of the data points that lies below this 75th percentile point will have value equal to or less than 1486143.
- For the high-level description purpose:
    - we can label all those values that lie between 0 and 25th percentile value as `low_population_range`.
    - all the values that lie between 25th percentile value and 75th percentile value as `medium_population_range`.
    - all values that lie between 75th percentile value to the maximum reported value can be termed as `high_population_range`.

In [ ]:
# Define the labels and conditions
conditions = [
    (data['population'] <= data['population'].quantile(0.25)),
    (data['population'] > data['population'].quantile(0.25)) & (data['population'] <= data['population'].quantile(0.75)),
    (data['population'] > data['population'].quantile(0.75))
]

labels = ['low_population_range', 'medium_population_range', 'high_population_range']

# Create a new column with the labels
data['population_range'] = np.select(conditions, labels, default='unknown')
data.drop(['population'], axis=1, inplace=True)   # Remove the column 'population' because we are using 'population_range' in place of that.
# Display the first few rows of the DataFrame with the new column
data.head(2)

In [ ]:
print(data["suicides/100k pop"].value_counts())

In [ ]:
print("Unique values in the column \"suicides/100k pop\":\n", data["suicides/100k pop"].unique())
print("Number of unique values:", data["suicides/100k pop"].nunique())
print("--------------------------------------------")
print("Unique values in the column \"country-year\":\n", data["country-year"].unique())
print("Number of unique values:", data["country-year"].nunique())
print("--------------------------------------------")

- As we are already using 'population' and 'suicides_no' attributes in their generalized form, we can remove the dimension "suicides/100k pop" from our dataset.
- Similarly, we are using distinct values in 'country' and 'year' dimension, therefore, we will drop the dimension 'country-year'.

In [ ]:
data.drop(['suicides/100k pop'], axis=1, inplace=True)   # Remove the mentioned column
data.drop(['country-year'], axis=1, inplace=True)   # Remove the mentioned column
data.drop(['gdp_per_capita ($)'], axis=1, inplace=True)

In [ ]:
data[" gdp_for_year ($) "].describe().apply(lambda x: format(x, 'f')) # Suppress Scientific Notation

- We can derive from the describe() that, the minimum reported gdp_per_year value is 46,919,625\\$. The maximum reported population is 18,120,714,000,000\\$.
- At 25th percentile, the reported gdp_per_year value is 8,985,352,832\\$. This means that 25 percent of data that lies below this 25th percentile point will have value equal to or less than 8,985,352,832\\$.
- At 50th percentile, the gdp_per_year value is 48,114,688,201\\$. This means half of the data points below 50th percentile point will have value equal to or less than 48,114,688,201\\$.
- At 75th percentile, the gdp_per_year value is 260,202,429,150\\$. This means that 75\% of the data points that lies below this 75th percentile point will have value equal to or less than 260,202,429,150\\$. 
- For the high-level description purpose:
    - we can label all those values that lie between 0 and 25th percentile value as `low_income_range`.
    - all the values that lie between 25th percentile value and 75th percentile value as `medium_income_range`.
    - all values that lie between 75th percentile value to the maximum reported value can be termed as `high_income_range`.

In [ ]:
# Define the labels and conditions
conditions = [
    (data[' gdp_for_year ($) '] <= data[' gdp_for_year ($) '].quantile(0.25)),
    (data[' gdp_for_year ($) '] > data[' gdp_for_year ($) '].quantile(0.25)) & (data[' gdp_for_year ($) '] <= data[' gdp_for_year ($) '].quantile(0.75)),
    (data[' gdp_for_year ($) '] > data[' gdp_for_year ($) '].quantile(0.75))
]

labels = ['low_income_range', 'medium_income_range', 'high_income_range']

# Create a new column with the labels
data['gdp_per_year_income_range'] = np.select(conditions, labels, default='unknown')
data.drop([' gdp_for_year ($) '], axis=1, inplace=True)   # Remove the column 'gdp+per_year ($)' because we are using 'gdp_per_year_income_range' in place of that.
# Display the first few rows of the DataFrame with the new column
data.head(2)

In [ ]:
print(data["generation"].value_counts())

In [ ]:
print("Unique values:\n", data["generation"].unique())
print("--------------------------------------------")
print("Number of unique values:", data["generation"].nunique())

In [ ]:
print(data.info)

In [ ]:
#Delete unnecessary variabels after Preprocessing stesps
del conditions, duplicate, labels

<a name = Section4></a>
#### **4. BUC Implementation**

- The cell below contains a function to encode categorical attributes to numerical.

In [ ]:
class preprocess_df:
  '''
  Class to preprocess DataFrame
  '''
  def encode_attributes(self, input_df, column_indices):
    transformed_dicts_ls = []
    transformed_df = input_df.copy(deep = True)
    column_names = transformed_df.columns.tolist()
    for col_iter in column_indices:
      temp_dict = {}
      temp_key = 0
      temp_ls = []
      column_name = column_names[col_iter]
      for col in transformed_df.iloc[:,col_iter].tolist():
        if col not in [*temp_dict.keys()]:
          temp_dict[col] = temp_key
          temp_key += 1
        temp_ls.append(temp_dict[col])
      dict_inv = {v:k for k,v in temp_dict.items()}
      transformed_dicts_ls.append(dict_inv)
      transformed_df[column_name] = temp_ls
    return transformed_df, transformed_dicts_ls

#### When the entire dataframe fits into the main memory
- Below cell contains the class for BUC implementation when the dataframe fits into the main memory.
- Most of the variable names are exactly as indicated in the paper.
- Counting sort is used as prescribed in the paper.

In [ ]:
class buc:
    '''
    Class for implementing BUC
    '''
    def __init__(self, df, column_enc_dicts_ls, minsup):
        self.numDims = df.shape[1]
        self.cardinality = []
        self.minsup = minsup
        self.output_df = None
        self.datacounts = [[]] * df.shape[1]
        self.attribute_ls = ["*"] * df.shape[1]
        self.debug_counter = 0
        self.output_dict = {}
        self.column_enc_dicts_ls = column_enc_dicts_ls

    def counting_sort(self, array_a, df_idx_ls):
      '''
      Inputs 
      array_a: List to be sorted
      df_idx_ls: Index list corresponding to the array_a. For example: DataFrame indices corresponding to array_a.
      Output
      idx_ls: Order in which df_idx_ls should be arranged so that array_a is in the sorted order i.e argsorting array_a
      '''
      array_c = [0]*(max(array_a) + 1)
      idx_ls = [-1] * (len(array_a))

      for i in range(0, len(array_a)):
        array_c[array_a[i]] += 1

      for i in range(0, len(array_c) - 1):
        array_c[i+1] = array_c[i] + array_c[i+1]

      for i in range(len(array_a) - 1, -1, -1):
        array_c[array_a[i]] = array_c[array_a[i]] - 1
        idx = array_c[array_a[i]]
        idx_ls[idx] = df_idx_ls[i]

      return idx_ls


    def partition(self, input_df, d):
        '''
        Implements partitioning logic i.e sorts the input dataframe and populates self.datacounts
        Inputs:
        input_df: Input DataFrame
        d: column number based on which sorting is performed
        Output:
        input_df: DataFrame which is sorted according to the specified column
        '''
        #Sorting the dataframe
        temp_counter_dict = {}
        sorted_idx = self.counting_sort(input_df.iloc[:,d].tolist(), input_df.index.tolist())
        input_df = input_df.reindex(sorted_idx)
        #Populating self.datacounts
        for attribute in input_df.iloc[:,d].tolist():
            temp_counter_dict[attribute] = temp_counter_dict.get(attribute, 0) + 1
        self.datacounts[d] = [*temp_counter_dict.values()]
        return input_df

    def buc_implementation(self, input, dim):
        '''
        Function to implement BUC as indicated in the original paper. 
        Populates self.output_dict which is the output dictionary.
        NOTE:Most of the variable names are exactly as indicated in the original paper.
        Input
        input: Input DataFrame
        dim: Starting column for performing aggregation
        '''
        self.debug_counter += 1
        if tuple(self.attribute_ls) in [*self.output_dict.keys()]:
          print(f"Error!!")
        self.output_dict[tuple(self.attribute_ls)] = input.shape[0]
       
        for d in range(dim, self.numDims,1):
            bigc = input.iloc[:,d].nunique()
            input = self.partition(input, d)
            k = 0
            for i in range(0, bigc, 1):
                smallc = self.datacounts[d][i]
                if smallc >= self.minsup:
                    self.attribute_ls[d] = self.column_enc_dicts_ls[d][input.iloc[k,d]]
                    self.buc_implementation(input.iloc[k:k+smallc,:], dim=d+1)
                k += smallc
            self.attribute_ls[d] = "*"

In [ ]:
def format_output(output_dict):
    '''
    Function that takes the output dictionary and converts it into a dataframe for readability
    '''
    output_dict_transformed = {}
    columns_ls = input_df.columns.tolist()
    for column in columns_ls:
        output_dict_transformed[column] = []
        output_dict_transformed['count'] = []
    for tuple_key, value in output_dict.items():
        output_dict_transformed['count'].append(value)
        for tuple_key_iter in range(0,len(tuple_key)):
            output_dict_transformed[columns_ls[tuple_key_iter]].append(tuple_key[tuple_key_iter])
    output_df = pd.DataFrame.from_dict(output_dict_transformed)
    columns_order = ['country', 'year', 'sex', 'age', 'generation', 'suicides_range', 'population_range', 'gdp_per_year_income_range', 'count']
    output_df = output_df.reindex(columns = columns_order)
    return output_df

- BUC Output when the entire dataframe fits into the main memory.

In [ ]:
#BUC Implementation: When the entire dataframe fits into the main memory.
minsup = 100
num_splits = 10
input_df = data
#Encode all of the categorical attributes into numerical form.
preprocess_obj = preprocess_df()
transformed_df, column_enc_dicts_ls = preprocess_obj.encode_attributes(input_df, [*range(0,input_df.shape[1])]) #NOTE: This should be modified as required
buc_obj = buc(transformed_df, column_enc_dicts_ls, minsup)
buc_obj.buc_implementation(transformed_df, 0)
output_dict = buc_obj.output_dict
format_output(output_dict)

In [ ]:
del minsup, num_splits, input_df, preprocess_obj, transformed_df, column_enc_dicts_ls, buc_obj, output_dict

#### When the entire dataframe doesn't fit into the main memory
- The dataframe is partitioned into a certain number of chunks which are saved on the disk.
- External merge sort is implemented for partitioning.
- For external merge sort, we perform a k-way merge after sorting each individual list (assuming that we are given k lists).

In [ ]:
def external_merge_sort_implementation(chunk_paths, d, iteration_num, index_dict, file_path):
    '''
    Implements external merge sort i.e to sort k dataframes when all of them don't fit in the main memory.
    Given K chunks, the function replaces the unsorted chunks with their globally sorted versions.
    Inputs:
    chunk_paths: Locations of each chunk
    d: Column number on which sorting is to be performed
    iteration_num: Iteration number
    index_dict: Dictionary containing the indices in each chunked dataframe.
    Output:
    None
    '''
    sorted_chunk_paths = []
    index_ls = copy.deepcopy([*index_dict.values()])
    output_len_ls = copy.deepcopy([*index_dict.values()])
    index_ls = [len(i) for i in index_ls]
    output_len_ls = [len(i) for i in output_len_ls]
    output_len_idx = 0
    #Sorting each individual dataframe
    for i_iter, chunk_path in enumerate(chunk_paths):
        chunk_df = pd.read_pickle(chunk_path)
        if i_iter == 0:
            df_columns = chunk_df.columns
        chunk_df_ls = chunk_df.values.tolist()
        chunk_df_ls.sort(key = lambda x: (x[d]))
        sorted_chunk_path = f"./sorted_{chunk_path.split('/')[-1].split('.pkl')[0]}"
        sorted_chunk_df = pd.DataFrame(chunk_df_ls, columns=df_columns)
        sorted_chunk_df.to_pickle(sorted_chunk_path)
        sorted_chunk_paths.append(sorted_chunk_path)

    #Merging the sorted chunks by performing a K-way merge operation
    output_ls = []
    idxs_ls = [0] * len(chunk_paths)
    input_ls = []
    min_idx = -1
    debug_counter = 0
    sorted_chunks_path_cpy = copy.deepcopy(sorted_chunk_paths)
    while True:
        #First iteration
        if min_idx == -1:
            for sorted_chunk_path in sorted_chunks_path_cpy:
                df = pd.read_pickle(sorted_chunk_path)
                input_ls.append(df.values.tolist()[idxs_ls[i_iter]])
        else:
            try:
                df = pd.read_pickle(sorted_chunks_path_cpy[min_idx])
                input_ls.insert(min_idx, df.values.tolist()[idxs_ls[min_idx]])
            except Exception as err:
                print(f"Inside except block... {err}")
  
        input_arr = np.array(input_ls)
        min_idx = np.argmin(input_arr, axis=0)[d]
        output_ls.append(input_ls[min_idx])
        input_ls.pop(min_idx)
        idxs_ls[min_idx] += 1

        #Writing output
        if len(output_ls) >= output_len_ls[output_len_idx]:
            sorted_chunk_df = pd.DataFrame(output_ls, columns=df_columns)
            sorted_chunk_df.to_pickle(f"{file_path}{iteration_num}{output_len_idx}.pkl")
            output_ls = []
            output_len_idx += 1
        
        if idxs_ls[min_idx] > (index_ls[min_idx] -1):
            idxs_ls.pop(min_idx)
            sorted_chunks_path_cpy.pop(min_idx)
            index_ls.pop(min_idx)
            if len(input_ls) == 0:
                break 
            try:
                if min_idx > len(input_ls) - 1:
                    min_idx -= 1
                    input_ls.pop(min_idx)
                else:
                    input_ls.pop(min_idx)
            except Exception as err:
                print(f"{err}")
                
    #Remove sorted_chunk_paths
    for sorted_chunk in sorted_chunk_paths:
        os.remove(sorted_chunk)

- Below cell contains the code for implementing the BUC class when the entire dataframe doesn't fit into the main memory.

In [ ]:
#BUC implementation
class buc_external:
    '''
    Class for implementing BUC
    '''
    def __init__(self, df, column_enc_dicts_ls, minsup):
        self.numDims = df.shape[1]
        self.cardinality = []
        self.minsup = minsup
        self.output_df = None
        self.datacounts = [[]] * df.shape[1]
        self.attribute_ls = ["*"] * df.shape[1]
        self.debug_counter = 0
        self.output_dict = {}
        self.column_enc_dicts_ls = column_enc_dicts_ls
        self.file_path = './dfs/iter_run_'
        os.makedirs(f"./dfs", exist_ok=True)

    def counting_sort(self, array_a, df_idx_ls):
      '''
      Inputs 
      array_a: List to be sorted
      df_idx_ls: Index list corresponding to the array_a. For example: DataFrame indices corresponding to array_a.
      Output
      idx_ls: Order in which df_idx_ls should be arranged so that array_a is in the sorted order.
      '''
      array_c = [0]*(max(array_a) + 1)
      idx_ls = [-1] * (len(array_a))

      for i in range(0, len(array_a)):
        array_c[array_a[i]] += 1

      for i in range(0, len(array_c) - 1):
        array_c[i+1] = array_c[i] + array_c[i+1]

      for i in range(len(array_a) - 1, -1, -1):
        array_c[array_a[i]] = array_c[array_a[i]] - 1
        idx = array_c[array_a[i]]
        idx_ls[idx] = df_idx_ls[i]

      return idx_ls

    def partition(self, iteration_num, num_splits, index_dict, d):
        '''
        Function to perform partitioning, external merge sort is used here.
        Inputs:
        iteration_num: Iteration number.
        num_splits: No. of chunks into which the input file is divided.
        index_dict: Dictionary containing the indices in each chunked dataframe.
        d: Attribute on which sorting is to be performed.
        '''
        chunk_paths_ls = []
        for i_iter in range(0, num_splits):
            if os.path.exists(f"{self.file_path}{iteration_num}{i_iter}.pkl"):
                chunk_paths_ls.append(f"{self.file_path}{iteration_num}{i_iter}.pkl")
                
        external_merge_sort_implementation(chunk_paths_ls, d, iteration_num, index_dict, self.file_path)
        
        input_df = pd.DataFrame()
        for i_iter in range(0, num_splits):
            if os.path.exists(f"{self.file_path}{iteration_num}{i_iter}.pkl"):
                df = pd.read_pickle(f"{self.file_path}{iteration_num}{i_iter}.pkl")
                input_df = pd.concat([input_df, df])
        
        #Populating self.datacounts
        temp_counter_dict = {}
        for attribute in input_df.iloc[:,d].tolist():
            temp_counter_dict[attribute] = temp_counter_dict.get(attribute, 0) + 1
        self.datacounts[d] = [*temp_counter_dict.values()]
        return None
                    
        
    def compute_aggregate(self, iteration_num, num_splits):
        '''
        Function to find the count i.e the total number of rows in all the chunks combined.
        '''
        count = 0
        for i_iter in range(0,num_splits):
            if os.path.exists(f"{self.file_path}{iteration_num}{i_iter}.pkl"):
                df = pd.read_pickle(f"{self.file_path}{iteration_num}{i_iter}.pkl")
                count += df.shape[0]
        return count

    
    def find_bigc(self, iteration_num, num_splits, d): 
        '''
        Bigc refers to the cardinality of the dth attribute in the dataframe
        '''
        computed_values = []
        bigc = 0
        for i_iter in range(0, num_splits):
            if os.path.exists(f"{self.file_path}{iteration_num}{i_iter}.pkl"):
                df = pd.read_pickle(f"{self.file_path}{iteration_num}{i_iter}.pkl")
                for attribute_name in df.iloc[:,d].unique().tolist():
                    if attribute_name not in computed_values:
                        computed_values.append(attribute_name)
                        bigc += 1
        return bigc

    def split_input(self, slice_range, iteration_num, num_splits):
        '''
        Function to split the input dataframe into a specified number of chunks. 
        Chunks in iteration 't' are obtained by taking the correct indices from iteration 't-1'
        '''
        #Delete previous files of current iteration
        for i_iter in range(0, num_splits):
            if os.path.exists(f"{self.file_path}{iteration_num}{i_iter}.pkl"):
                os.remove(f"{self.file_path}{iteration_num}{i_iter}.pkl")
                            
        index_dict_t = {} #Stores the indices that are present in each dictionary
        #Indicates the very first iteration
        if not os.path.exists(f"{self.file_path}{iteration_num-1}0.pkl"):
            input_df = transformed_df
            split_df = np.array_split(input_df, num_splits)
            start_idx = 0
            for i_iter, df in enumerate(split_df):
                # print(f"DF after splitting: {df}")
                df.to_pickle(f"{self.file_path}{iteration_num}{i_iter}.pkl")
                index_dict_t[i_iter] = [*range(start_idx, start_idx + df.shape[0])]
                start_idx += df.shape[0]   
            with open(f"{self.file_path}_dict_{iteration_num}.pkl", "wb") as fp:
                pickle.dump(index_dict_t, fp)
        else:  
            if (slice_range[1] - slice_range[0]) < num_splits:
                df_lengths_t = [slice_range[1] - slice_range[0]]
                df_lengths_t.extend([0]*(num_splits-1))
            else:
                df_lengths_t = [math.floor((slice_range[1] - slice_range[0])/num_splits)]*num_splits
                df_lengths_t[-1] += (slice_range[1] - slice_range[0])%num_splits
            df_ranges_t_ls = []
            start_idx = slice_range[0]
            for df_length in df_lengths_t:
                if df_length != 0:
                    df_ranges_t_ls.append([*range(start_idx,start_idx + df_length)])
                    start_idx += df_length
            del start_idx 
            
            with open(f"{self.file_path}_dict_{iteration_num-1}.pkl", "rb") as fp:
                index_dict_tminus = pickle.load(fp)

            start_dict_idx = 0
            index_dict_t = {}
            for i_iter, df_range_t in enumerate(df_ranges_t_ls):
                start_df_idx = 0
                df = pd.DataFrame()
                for df_tminus_name, df_tminus_range in index_dict_tminus.items():
                    common_elements = [i for i in df_range_t for j in df_tminus_range if i == j]
                    common_elements = [i-start_df_idx for i in common_elements]
                    if len(common_elements) > 0:
                        temp_df = pd.read_pickle(f"{self.file_path}{iteration_num-1}{df_tminus_name}.pkl")
                        df = pd.concat([df, temp_df.iloc[common_elements,:]])
                    start_df_idx += len(df_tminus_range)
                #Writing output pickle files
                df.to_pickle(f"{self.file_path}{iteration_num}{i_iter}.pkl")
                index_dict_t[i_iter] = [*range(start_dict_idx, start_dict_idx + df.shape[0])]
                start_dict_idx += df.shape[0]
                #Writing the output dictionary
                with open(f"{self.file_path}_dict_{iteration_num}.pkl", "wb") as fp:
                    pickle.dump(index_dict_t, fp)
        return index_dict_t

    def populate_attribute_ls(self, k, d, iteration_num):
        '''
        Function to populate items in the output dictionary
        '''
        with open(f"{self.file_path}_dict_{iteration_num}.pkl", "rb") as fp:
            index_dict_t = pickle.load(fp)

        k_idx = k
        for df_name, df_idxs in index_dict_t.items():
            if k in df_idxs:
                df = pd.read_pickle(f"{self.file_path}{iteration_num}{df_name}.pkl")
                self.attribute_ls[d] = self.column_enc_dicts_ls[d][df.iloc[k_idx,d]]
                return None
            k_idx -= len(df_idxs)
        
        
    
    def buc_implementation(self, slice_range, dim, iteration_num, num_splits):
        '''
        Function to implement BUC when it is assumed that the dataframe doesn't fit in the main memory. 
        Populates self.output_dict which is the output dictionary.
        Inputs
        slice_range: Rows of the dataframe which have to sliced at iteration 't'.
        dim: Starting column for performing aggregation.
        iteration_num: Iteration number.
        num_splits: No.of chunks into which the input dataframe is to be divided.
        '''
        self.debug_counter += 1
        #Split the input into chunks, chunks at iteration 't' are computed from chunks at iteration 't-1'
        index_dict = self.split_input(slice_range, iteration_num, num_splits)
        #Compute count
        aggregate = self.compute_aggregate(iteration_num, num_splits)
        self.output_dict[tuple(self.attribute_ls)] = aggregate
        
        for d in range(dim, self.numDims,1):
            bigc = self.find_bigc(iteration_num, num_splits, d)
            #External partitioning: Calls merge sort
            self.partition(iteration_num, num_splits, index_dict, d)
            k = 0
            for i in range(0, bigc, 1):
                smallc = self.datacounts[d][i]
                if smallc >= self.minsup:
                    self.populate_attribute_ls(k, d, iteration_num)
                    self.buc_implementation(slice_range = [k,k+smallc], dim=d+1, iteration_num=iteration_num+1, num_splits=num_splits)
                k += smallc
            self.attribute_ls[d] = "*"

In [ ]:
minsup = 100
input_df = data
num_splits = 20
preprocess_obj = preprocess_df()
transformed_df, column_enc_dicts_ls = preprocess_obj.encode_attributes(input_df, [*range(0,input_df.shape[1])]) #NOTE: This should be modified as required
buc_obj = buc_external(transformed_df, column_enc_dicts_ls, minsup)
buc_obj.buc_implementation([0,transformed_df.shape[0]], 0, 0, num_splits)
output_dict = buc_obj.output_dict
format_output(output_dict)

#### a) A plot of minsup vs. runtime, keeping allotted memory fixed.

- Here it is assumed that the dataframe fits into the main memory.
- The value of minsup is varied between 90 and 200 in steps of 10.

In [ ]:
num_splits = 10
input_df = data
#Encode all of the categorical attributes into numerical form.
preprocess_obj = preprocess_df()
transformed_df, column_enc_dicts_ls = preprocess_obj.encode_attributes(input_df, [*range(0,input_df.shape[1])]) #NOTE: This should be modified as required
minsup_ls = []
time_taken_ls = []
for minsup in range(90, 200, 10):
    minsup_ls.append(minsup)
    start_time = time.time()
    buc_obj = buc(transformed_df, column_enc_dicts_ls, minsup)
    buc_obj.buc_implementation(transformed_df, 0)
    time_taken_ls.append(time.time() - start_time)
del minsup, num_splits, input_df, preprocess_obj, transformed_df, column_enc_dicts_ls

In [ ]:
plt.scatter(minsup_ls, time_taken_ls)
plt.xlabel(f"Minimum Support")
plt.ylabel(f"Runtime (seconds)")
plt.title(f"Plot of Minimum support vs Runtime(seconds)")
del minsup_ls, time_taken_ls

#### b) A plot of allotted memory vs. runtime, keeping minsup fixed.

- The variation in the allocated main memory is indicated by varying the number of chunks into which the original dataframe is divided i.e greater the number of chunks lesser is the main memory that is allocated.
-  NOTE: The dataframe is restricted to 10K rows to speed up the execution.

In [ ]:
minsup = 200
input_df = data.loc[0:10000,:]
preprocess_obj = preprocess_df()
transformed_df, column_enc_dicts_ls = preprocess_obj.encode_attributes(input_df, [*range(0,input_df.shape[1])]) #NOTE: This should be modified as required
chunks_ls = []
time_taken_ls = []
for num_splits in range(30,50,5):
    chunks_ls.append(num_splits)
    start_time = time.time()
    buc_obj = buc_external(transformed_df, column_enc_dicts_ls, minsup)
    buc_obj.buc_implementation([0,transformed_df.shape[0]], 0, 0, num_splits)
    time_taken_ls.append(time.time() - start_time)
    # print(f"{time_taken_ls = }")
del minsup, num_splits, input_df, preprocess_obj, transformed_df, column_enc_dicts_ls

In [ ]:
plt.plot(chunks_ls, time_taken_ls)
plt.xlabel(f"No.of chunks into which main memory is divided")
plt.ylabel(f"Time taken in seconds")
plt.title(f"Plot of Number of chunks(indication of main memory) vs Runtime (seconds)")

----

## PART B

<a name = Section1></a>
#### **1. Data Acquisition and Description**

Lets analyze the dataset and identify what attributes require generalization/categorization

In [ ]:
property_data = pd.read_excel('./data/cleaned_data.xlsx')      # Load the Excel dataset
print('Shape of the dataset:', property_data.shape)
property_data.head(3)

- We have 143,708 records and 32 attributes.
- In our records, we have variety of data including nominal data, binomial data, numerical data and textual data.

In [ ]:
property_data.info() # Display basic information about the dataset

- Here we show the data type of our various attributes.

Always check for:
1. duplicate values in rows - delete duplicate rows
2. missing values in column

In [ ]:
duplicate = property_data[property_data.duplicated()]     # Selecting duplicate rows except first occurrence based on all columns
duplicate

In [ ]:
print(property_data.isnull().sum())      # Check for missing values

- We observe that there are no missing and duplicate values.

In [ ]:
property_data.columns # columns in our dataframe

<a name = Section2></a>
#### **2. Data Analysis and AOI**

Now, let's one by one, analyze the 32 dimensions and determine for which dimension, we need to perform Attribute Oriented Induction (AOI) for generalization/categorization.

Data generalization summarizes data by replacing relatively low-level values with higher-level concepts, or by reducing the number of dimensions to summarize data in concept space involving fewer dimensions.

----

In [ ]:
print("Unique values in coloumn 'Property_Name':\n", property_data["Property_Name"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Property_Name"].nunique())

In [ ]:
print("Unique values in coloumn 'Property_id':\n", property_data["Property_id"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Property_id"].nunique())

In [ ]:
print("Unique values in coloumn 'builder_id':\n", property_data["builder_id"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["builder_id"].nunique())

In [ ]:
print("Unique values in coloumn 'Builder_name':\n", property_data["Builder_name"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Builder_name"].nunique())

Let's drop all these columns as they don't seem to provide any significant information, also they have a lot of distinct values.

In [ ]:
property_data.drop(['Property_Name'], axis=1, inplace=True)     # Remove the mentioned column
property_data.drop(['Property_id'], axis=1, inplace=True)     # Remove the mentioned column
property_data.drop(['builder_id'], axis=1, inplace=True)     # Remove the mentioned column
property_data.drop(['Builder_name'], axis=1, inplace=True)     # Remove the mentioned column

----

In [ ]:
print("Unique values in coloumn 'Property_type':\n", property_data["Property_type"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Property_type"].nunique())

- We will use the values of 'Property_type' dimension as it is because it is already characterized by five distinct values.

In [ ]:
print("Unique values in coloumn 'Property_status':\n", property_data["Property_status"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Property_status"].nunique())

- We will use the values of 'Property_status' dimension as it is because it is characterized by 2 distinct values.

----

In [ ]:
print("Unique values in coloumn 'Price_per_unit_area':\n", property_data["Price_per_unit_area"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Price_per_unit_area"].nunique())

- Let's remove the dimension 'Price_per_unit_area' as we already have the dimensions 'Price' and 'Size' that we will generalize.

In [ ]:
property_data.drop(['Price_per_unit_area'], axis=1, inplace=True)     # Remove the mentioned column

In [ ]:
print("Unique values in coloumn 'Price':\n", property_data["Price"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Price"].nunique())

In [ ]:
property_data['Price'].head()

- However, it seems that the data type of **Price** is object. Let's change that to float, so that we can describe the data.

In [ ]:
# convert object type to str, removing comma and then convert str dtype to float
property_data['Price'] = property_data['Price'].astype('str') 
property_data['Price'] = property_data['Price'].str.replace(',','')
property_data['Price'] = property_data['Price'].astype(float)

property_data['Price'].head()

In [ ]:
property_data['Price'].describe().apply(lambda x: format(x, 'f')) # suppress scientific notation

- We can derive from the describe() that, the minimum value of Price is 100000. The maximum value of Price is 800000000. 
At 25th percentile, the value of Price is value is 5259437. This means that 25 percent of data that lies below this 25th percentile point will have value equal to or less than 5259437. 
- At 50th percentile, the value of Price is 8500000. This means half of the data points below 50th percentile point will have value equal to or less than 8500000.
- At 75th percentile, the price value is 15498000. This means that 75% of the data points that lies below this 75th percentile point will have value equal to or less than 15498000. 
- For the high-level description purpose, we label all those values below 25th percentile point as `low_price_range`.
- We label all those values above low_price_range and below the value at 75th percentile as `medium_price_range`.
- Similarly, all values that lie between 75th percentile value to the maximum reported value can be termed as `high_price_range`.

In [ ]:
# Define the labels and conditions
conditions = [
    (property_data["Price"] <= property_data["Price"].quantile(0.25)),
    (property_data["Price"] > property_data["Price"].quantile(0.25)) & (property_data["Price"] <= property_data["Price"].quantile(0.75)),
    (property_data["Price"] > property_data["Price"].quantile(0.75))
]

labels = ['low_price_range', 'mid_price_range', 'high_price_range']

# Create a new column with the labels
property_data["Price_range"] = np.select(conditions, labels, default='unknown')
property_data.drop(['Price'], axis=1, inplace=True)   # Remove the column 'Price' because we are using 'Price_range' in place of that.
# Display the first few rows of the DataFrame with the new column
property_data.head(2)

In [ ]:
print("Unique values in coloumn 'Size':\n", property_data["Size"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Size"].nunique())

In [ ]:
property_data["Size"].head(2)

- However, it seems that the data type of **Size** is object. Let's change that to float, so that we can describe the data.

In [ ]:
# convert object type to str, removing comma and letters and then convert str dtype to int
property_data['Size'] = property_data['Size'].astype('str') 
property_data['Size'] = property_data['Size'].str.replace(',','') 
property_data['Size'] = property_data['Size'].str.extract('(^[^\s]+)')
property_data['Size'] = property_data['Size'].astype(int)
property_data['Size'].head()

In [ ]:
property_data["Size"].describe()

- We can derive from the describe() that, the minimum value of Size is 100 sq. ft.. The maximum value of Size is 90000 sq. ft.. 
At 25th percentile, the value of Size is value is 720 sq. ft. This means that 25 percent of the entire data that lies below this 25th percentile point will have value equal to or less than 720 sq. ft. 
- At 50th percentile, the value of Size is 1076 sq ft. This means half of the data points below 50th percentile point will have value equal to or less than 1076.
- At 75th percentile, the Size value is 1516. This means that 75% of the data points that lies below this 75th percentile point will have value equal to or less than 1516 sq ft. 
- For the high-level description purpose, let's create bins grouping certain sizes together.

In [ ]:
bins = [i*100 for i in range(int(property_data['Size'].quantile(0.75))//100)]
bins.append(int(property_data['Size'].max()))
labels = ["{}-{}".format(bins[i], bins[i+1]) for i in range(len(bins) - 1)]
# print(bins)
labels[-1]=">{}".format(bins[-2])
# print(labels)

property_data["Size_range"] = pd.cut(property_data["Size"], bins=bins, labels=labels)

property_data.drop(['Size'], axis=1, inplace=True)   # Remove the column 'Size' because we are using 'Size_range' in place of that.
# Display the first few rows of the DataFrame with the new column
property_data.head(2)

In [ ]:
print("Unique values in coloumn 'Size_range':\n", property_data["Size_range"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Size_range"].nunique())

----

In [ ]:
print("Unique values in coloumn 'Posted_On':\n", property_data["Posted_On"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Posted_On"].nunique())

- Let's use AOI to categorize the values of 'Posted_On' dimension into the following values:
    - `< an hour`:               This encompasses all the postings done within an hour
    - `>= hour and < 1 day`:     This encompasses all the postings done between an hour (include) and a day (exclude)    
    - `=> 1 day and < 1 month`:  This encompasses all the postings done between one day (include) and a month (exclude)    
    - `=> 1 month and < 1 year`: This encompasses all the postings done between one month (include) and a year (exclude)    
    - `=> 1 year and < 5 year`:  This encompasses all the postings done between one year (include) and five years (exclude)  

In [ ]:
property_data.loc[property_data["Posted_On"].str.contains('minute'), "Posted_On_range"] = '< an hour'
property_data.loc[property_data["Posted_On"].str.contains('hour'), "Posted_On_range"] = '=> hour and < 1 day'
property_data.loc[property_data["Posted_On"].str.contains('day'), "Posted_On_range"] = '=> 1 day and < 1 month'
property_data.loc[property_data["Posted_On"].str.contains('month'), "Posted_On_range"] = '=> 1 month and < 1 year'
property_data.loc[property_data["Posted_On"].str.contains('year'), "Posted_On_range"] = '=> 1 year and < 5 year'

property_data.head(2)

In [ ]:
print(property_data["Posted_On_range"].value_counts())
print(".......................................................................")
print("Unique values", property_data["Posted_On_range"].nunique())

In [ ]:
property_data.drop(['Posted_On'], axis=1, inplace=True)   # Remove the column 'Posted_On' because we are using 'Posted_On_range' in place of that.

----

In [ ]:
print("Unique values in coloumn 'Project_URL':\n", property_data["Project_URL"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Project_URL"].nunique())

In [ ]:
property_data.drop(['Project_URL'], axis=1, inplace=True)     # Dropping the dimension

----

In [ ]:
print("Unique values in coloumn 'Property_building_status':\n", property_data["Property_building_status"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Property_building_status"].nunique())

- We will use the values of 'Property_building_status' dimension as it is, because it is generalized into 3 distinct values.

----

In [ ]:
print("Unique values in coloumn 'City_id':\n", property_data["City_id"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["City_id"].nunique())

In [ ]:
print("Unique values in coloumn 'City_name':\n", property_data["City_name"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["City_name"].nunique())

- As we can see, 'City_id' is just a unique ID corresponding to a unique 'City_name', that is both the dimensions represent the same entity.
- Therefore, we can remove any one of them.
- Let's remove 'City_id' as using 'City_name' is much more intuitive for humans.

In [ ]:
property_data.drop(['City_id'], axis=1, inplace=True)     # Remove the mentioned column

----

In [ ]:
print("Unique values in coloumn 'No_of_BHK':\n", property_data["No_of_BHK"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["No_of_BHK"].nunique())

- We will use the values of 'No_of_BHK' dimension as it is because it is already characterized by 17 distinct values.

----

In [ ]:
print("Unique values in coloumn 'Locality_ID':\n", property_data["Locality_ID"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Locality_ID"].nunique())

In [ ]:
print("Unique values in coloumn 'Locality_Name':\n", property_data["Locality_Name"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Locality_Name"].nunique())

In [ ]:
print("Unique values in coloumn 'Longitude':\n", property_data["Longitude"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Longitude"].nunique())

In [ ]:
property_data.drop(['Longitude'], axis=1, inplace=True)   # Remove the column 'Longitude'
property_data.drop(['Latitude'], axis=1, inplace=True)   # Remove the column 'Latitude'

In [ ]:
print("Unique values in coloumn 'Sub_urban_ID':\n", property_data["Sub_urban_ID"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Sub_urban_ID"].nunique())

In [ ]:
print("Unique values in coloumn 'Sub_urban_name':\n", property_data["Sub_urban_name"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Sub_urban_name"].nunique())

In [ ]:
property_data.drop(['Sub_urban_ID'], axis=1, inplace=True)     # Remove the 'Sub_urban_ID' column as we already have 'Sub_urban_name'

----

In [ ]:
print("Unique values in coloumn 'description':\n", property_data["description"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["description"].nunique())

In [ ]:
property_data.drop(['description'], axis=1, inplace=True)     # Remove the mentioned column

----

In [ ]:
print("Unique values in coloumn 'is_furnished':\n", property_data["is_furnished"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["is_furnished"].nunique())

- Dimension 'is_furnished' has 3 distinct values and therefore we will use this as it is

In [ ]:
print("Unique values in coloumn 'listing_domain_score':\n", property_data["listing_domain_score"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["listing_domain_score"].nunique())

- Listing domain score is an important dimension that we want to keep. Also, it has 18 distinct values that we can use as it is.

----

In [ ]:
print("Unique values in coloumn 'is_plot':\n", property_data["is_plot"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["is_plot"].nunique())

In [ ]:
property_data[property_data["is_plot"] == True].head(2)

- Dimension 'is_plot' has 2 distinct values and it indicates that whether a property is a 'Residential Plot' or not.
- We can see from above that 'is_plot'==True can also be represented by value 'Residential Plot' in the dimension 'Property_type'. Given this correlation, we can remove the dimension 'is_plot'.

In [ ]:
property_data.drop(['is_plot'], axis=1, inplace=True)     # Remove the mentioned column

----

In [ ]:
print("Unique values in coloumn 'is_RERA_registered':\n", property_data["is_RERA_registered"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["is_RERA_registered"].nunique())

- The dimension 'is_RERA_registered' has 2 distinct values and is already in a generalized form.

----

In [ ]:
print("Unique values in coloumn 'is_Apartment':\n", property_data["is_Apartment"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["is_Apartment"].nunique())

In [ ]:
property_data[property_data["is_Apartment"] == True].head(2)

- Dimension 'is_Apartment' has 2 distinct values and it indicates that whether a property is an 'Apartment' or not.
- We can see from above that 'is_Apartment'==True can also be represented by value 'Apartment' in the dimension 'Property_type'. Given this correlation, we can remove the dimension 'is_Apartment'.

In [ ]:
property_data.drop(['is_Apartment'], axis=1, inplace=True)     # Remove the mentioned column

----

In [ ]:
print("Unique values in coloumn 'is_ready_to_move':\n", property_data["is_ready_to_move"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["is_ready_to_move"].nunique())

In [ ]:
property_data[property_data["is_ready_to_move"] == True].head(2)

- Dimension 'is_ready_to_move' has 2 distinct values and it indicates that whether a property status is 'is_ready_to_move' or not.
- We can see from above that 'is_ready_to_move'==True can also be represented by value 'Ready to move' in the dimension 'Property_status'. Given this correlation, we can remove the dimension 'is_ready_to_move'.

In [ ]:
property_data.drop(['is_ready_to_move'], axis=1, inplace=True)     # Remove the mentioned column

----

In [ ]:
print("Unique values in coloumn 'is_commercial_Listing':\n", property_data["is_commercial_Listing"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["is_commercial_Listing"].nunique())

- Dimension 'is_commercial_Listing' has only one value and therefore can be removed.

In [ ]:
property_data.drop(['is_commercial_Listing'], axis=1, inplace=True)     # Remove the mentioned column

----

- Now let us analyze the columns 'is_Pentahouse' and 'is_studio'

In [ ]:
print("Unique values in coloumn 'is_PentaHouse':\n", property_data["is_PentaHouse"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["is_PentaHouse"].nunique())

In [ ]:
print("Unique values in coloumn 'is_studio':\n", property_data["is_studio"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["is_studio"].nunique())

- We know that both Pentahouse and Studio are a type of apartment.
- Let us see how we can perform AOI, by incorporating value 'Pentahouse' and 'Studio' in the dimension 'Property_type' thus reducing the total number of dimensions without losing out on important information.

In [ ]:
display(property_data.loc[(property_data['is_PentaHouse']==True) & (property_data['is_studio']==True) & (property_data['Property_type'].str.startswith('Apartment')), ['Property_type']])

- The above output shows that an apartment can never be both 'Pentahouse' and 'Studio' at the same time (which confirms to our intuitive understanding).

In [ ]:
display(property_data.loc[(property_data['is_PentaHouse']==False) & (property_data['is_studio']==False) & (property_data['Property_type'].str.startswith('Apartment')), ['Property_type']])

- The above output shows that an apartment need not be either a 'Pentahouse' or 'Studio'.

- Equipped with this knowledge, let's incorporate two more values in the dimension 'Property_type', i.e., `Pentahouse_Apartment` and `Studio_Apartment`.
- This will lead to overall 7 values in the dimension 'Property_type'.

In [ ]:
property_data.loc[property_data["is_PentaHouse"]==True, "Property_type"] = 'Pentahouse_Apartment'
property_data.loc[property_data["is_studio"]==True, "Property_type"] = 'Studio_Apartment'

- Let's see the value of dimension 'Property_type' if 'is_studio'==True

In [ ]:
property_data[property_data["is_studio"] == True].head(2)

- Similarly, let's see the value of dimension 'Property_type' if 'is_PentaHouse'==True

In [ ]:
property_data[property_data["is_PentaHouse"] == True].head(2)

- Let's verify the values of the dimension 'Property_type'.

In [ ]:
print("Unique values in coloumn 'Property_type':\n", property_data["Property_type"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Property_type"].nunique())

- Let's drop the dimensions 'is_PentaHouse' and 'is_studio', as they have been successfully incorporated.

In [ ]:
property_data.drop(['is_PentaHouse', 'is_studio'], axis=1, inplace=True)     # Remove the mentioned two columns

----

In [ ]:
print("Unique values in coloumn 'Listing_Category':\n", property_data["Listing_Category"].unique())
print(".......................................................................")
print("Number of unique values:", property_data["Listing_Category"].nunique())

- Dimension 'Listing_Category' has only one value and therefore can be removed.

In [ ]:
property_data.drop(['Listing_Category'], axis=1, inplace=True)     # Remove the mentioned column

----

In [ ]:
property_data.columns

In [ ]:
print('Shape of the dataset:', property_data.shape)

- As we can observe, after performing AOI, our dimension size has come down from 32 to 14.

----

- Final dataframe looks like:

In [ ]:
property_data.head(10)

----

----